# Automated ML on Heart Failure Dataset

Importing dependencies

In [1]:
import os
import joblib
import azureml.core
from azureml.core import Workspace, Experiment, Dataset, Environment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
from pprint import pprint # Used in printing automl model parameters
from azureml.core import Model # Used to get model information

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.26.0


## Initialize Workspace

Initialize a workspace object from persisted configuration. 

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-142998
aml-quickstarts-142998
southcentralus
3d1a56d2-7c81-4118-9790-f85d1acf0c77


## Create an Azure ML experiment

Create an [Experiment](https://docs.microsoft.com/en-gb/azure/machine-learning/concept-azure-machine-learning-architecture#experiment) to track all the runs in your workspace.

In [3]:
# Choose a name for the run history container in the workspace
experiment_name = 'heartfailure-automl'
experiment = Experiment(ws, experiment_name)

run = experiment.start_logging()

## Create or Attach an AmlCompute cluster

Create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for AutoML run

In [4]:
# Choose a name for your cluster
# Compute name should contain only letters, digits, hyphen and should be 2-16 charachters long
#cluster_name = "aml-cluster"
cluster_name = "project-automl"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print(f'{cluster_name} exists already')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    
    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    
    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
    
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

project-automl exists already
notebook142998 ComputeInstance Succeeded
project-automl AmlCompute Succeeded


## Dataset

### Overview

In this project, we are going to predict mortality due to heart failure with the use of AutoML. Heart failure is a common event caused by Cardiovascular diseases (CVDs), and it occurs when the heart cannot pump enough blood to meet the needs of the body.

The [Heart Failure Prediction](https://archive.ics.uci.edu/ml/datasets/Heart+failure+clinical+records) dataset is used as the training data for this task. It comprises of 299 heart failure patients and 12 features, which report clinical, body, and lifestyle information.

The task here is to train a binary classification model that predict the target column DEATH_EVENT, which indicates if the patient died or survived before the end of the follow-up period, based on the information provided by the other 11 columns (predictors). The time feature was dropped before training since we cannot get a time value for new patients after deployment. Prediction models based on these predictors, if accurate, can potentially be used to help hospitals in assessing the severity of patients with cardiovascular diseases.

In [5]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "Heart-Failure Dataset"
description_text = "Heart-Failure Dataset for Captone project"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/PeacePeters/Heart-Failure-Prediction-using-AzureML/main/heart_failure.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        # Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [6]:
# Review the dataset result
dataset.take(5).to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## AutoML Configuration

The AutoML settings are:
1. The model is a classification task to predict mortality caused by heart failure.
2. The primary metric used is AUC weighted, which is more appropriate than accuracy since the dataset is moderately imbalanced (67.89% negative elements and 32.11% positive elements). 
3. A cross validation of 5 folds rather than 3 is used which gives a better performance. 
4. A 30 minutes timeout is specified to constrain usage. 
5. The maximum number of iterations to be executed in parallel during training is set to 5 max concurrent iterations. 

In [7]:
# Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
    }

# Put your automl config here
automl_config = AutoMLConfig(
        compute_target=compute_target,
        task="classification",
        training_data=dataset,
        label_column_name="DEATH_EVENT",
        n_cross_validations=5,
        debug_log="automl_errors.log",
        **automl_settings
        )

In [8]:
# Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
heartfailure-automl,AutoML_35950e74-8857-40bf-a9d1-607581a39c1c,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

In [9]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
heartfailure-automl,AutoML_35950e74-8857-40bf-a9d1-607581a39c1c,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS

        69   StandardScalerWrapper XGBoostClassifier        0:00:52       0.9060    0.9162
        70   MinMaxScaler ExtremeRandomTrees                0:00:53       0.8775    0.9162
        71   StandardScalerWrapper RandomForest             0:00:55       0.9104    0.9162
        72   StandardScalerWrapper XGBoostClassifier        0:00:47       0.9070    0.9162
        73   StandardScalerWrapper XGBoostClassifier        0:00:50       0.9084    0.9162
        74   StandardScalerWrapper XGBoostClassifier        0:00:49       0.9054    0.9162
        75   MinMaxScaler ExtremeRandomTrees                0:00:48       0.7606    0.9162
        76   StandardScalerWrapper XGBoostClassifier        0:00:47       0.9037    0.9162
        77   MaxAbsScaler LightGBM                          0:00:48       0.8950    0.9162
        78   RobustScaler LightGBM                          0:00:49       0.9000    0.9162
        79   StandardScalerWrapper GradientBoosting         0:00:52       0.9081    0.9162

{'runId': 'AutoML_35950e74-8857-40bf-a9d1-607581a39c1c',
 'target': 'project-automl',
 'status': 'Completed',
 'startTimeUtc': '2021-04-18T08:03:07.142198Z',
 'endTimeUtc': '2021-04-18T08:42:17.086535Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'project-automl',
  'AMLSettingsJsonString': '{"path":null,"name":"heartfailure-automl","subscription_id":"3d1a56d2-7c81-4118-9790-f85d1acf0c77","resource_group":"aml-quickstarts-142998","workspace_name":"quick-starts-ws-142998","region":"southcentralus","compute_target":"project-automl","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"iterations":1000,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"test_size":0.

## Best Model

In [10]:
# Get best run and model
best_run, best_model = remote_run.get_output()

In [11]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
heartfailure-automl,AutoML_35950e74-8857-40bf-a9d1-607581a39c1c_102,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [33]:
print('Best Run Id: ' + best_run.id,
     'Best Model Name: ' + best_run.properties['model_name'])
print('\n AUC_weighted:', best_run_metrics['AUC_weighted'])

Best Run Id: AutoML_35950e74-8857-40bf-a9d1-607581a39c1c_102 Best Model Name: AutoML35950e748102

 AUC_weighted: 0.9229042081949059


In [12]:
# Get all metrics of the best run
best_run_metrics = best_run.get_metrics()

# Print all metrics of the best run
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

average_precision_score_macro 0.9133496361084446
f1_score_micro 0.8393785310734463
precision_score_micro 0.8393785310734463
balanced_accuracy 0.8138226744186046
recall_score_weighted 0.8393785310734463
average_precision_score_micro 0.9247512466618997
recall_score_macro 0.8138226744186046
precision_score_weighted 0.851982158953264
weighted_accuracy 0.8564669133875988
AUC_weighted 0.9229042081949059
matthews_correlation 0.6342729953440103
AUC_micro 0.9205290146509622
f1_score_macro 0.8072772141363057
accuracy 0.8393785310734463
precision_score_macro 0.8220723263515026
average_precision_score_weighted 0.9345240014199924
log_loss 0.36878216313098455
norm_macro_recall 0.6276453488372093
f1_score_weighted 0.8358834712120726
recall_score_micro 0.8393785310734463
AUC_macro 0.9229042081949059
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_35950e74-8857-40bf-a9d1-607581a39c1c_102/confusion_matrix
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_35950e74-8857-40bf-a9d1-60758

In [14]:
best_model._final_estimator

PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('83',
                                           Pipeline(memory=None,
                                                    steps=[('standardscalerwrapper',
                                                            <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7f538bc49e10>),
                                                           ('gradientboostingclassifier',
                                                            GradientBoostingClassifier(ccp_alpha=0.0,
                                                                                       criterion='mse',
                                                                                       init=None,
                                                                                       learning_rate=0.02154434690031882...
                                                                     

In [15]:
# Print detailed parameters of the fitted model
def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(best_model)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['83', '7', '54', '32', '86', '49', '53', '23'],
 'weights': [0.09090909090909091,
             0.09090909090909091,
             0.18181818181818182,
             0.09090909090909091,
             0.09090909090909091,
             0.09090909090909091,
             0.2727272727272727,
             0.09090909090909091]}

83 - standardscalerwrapper
{'class_name': 'StandardScaler',
 'copy': True,
 'module_name': 'sklearn.preprocessing._data',
 'with_mean': True,
 'with_std': False}

83 - gradientboostingclassifier
{'ccp_alpha': 0.0,
 'criterion': 'mse',
 'init': None,
 'learning_rate': 0.021544346900318822,
 'loss': 'devi

In [16]:
best_run.get_tags()

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":3,"CurrentNodeCount":4}',
 'mlflow.source.type': 'JOB',
 'mlflow.source.name': 'automl_driver.py',
 'ensembled_iterations': '[83, 7, 54, 32, 86, 49, 53, 23]',
 'ensembled_algorithms': "['GradientBoosting', 'ExtremeRandomTrees', 'XGBoostClassifier', 'XGBoostClassifier', 'GradientBoosting', 'LightGBM', 'LightGBM', 'RandomForest']",
 'ensemble_weights': '[0.09090909090909091, 0.09090909090909091, 0.18181818181818182, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.2727272727272727, 0.09090909090909091]',
 'best_individual_pipeline_score': '0.9167395256552233',
 'best_individual_iteration': '83',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True'}

## Model Deployment

In [17]:
# Register the model
myModel = best_run.register_model(model_path='outputs/model.pkl', model_name=experiment_name+"-model",
                   tags={'Training context':'AutoML', 'type': 'Classification'})
myModel

Model(workspace=Workspace.create(name='quick-starts-ws-142998', subscription_id='3d1a56d2-7c81-4118-9790-f85d1acf0c77', resource_group='aml-quickstarts-142998'), name=heartfailure-automl-model, id=heartfailure-automl-model:1, version=1, tags={'Training context': 'AutoML', 'type': 'Classification'}, properties={})

In [18]:
# List registered models to verify if model has been saved
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

heartfailure-automl-model version: 1
	 Training context : AutoML
	 type : Classification


hyperdrive-heart-failure-best-model version: 1
	 Training context : Parameterized SKLearn Estimator
	 type : Classification
	 AUC_weighted : 0.8333333333333334




In [19]:
best_run.download_file('outputs/model.pkl', './model.pkl')

# Download scoring file
best_run.download_file('outputs/scoring_file_v_1_0_0.py', './score.py')

best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'envFile.yml')

In [20]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

env = Environment.get(ws, "AzureML-AutoML")
inference_config = InferenceConfig(entry_script='./score.py', environment=env)

aci_config = AciWebservice.deploy_configuration(
                                               cpu_cores=1,
                                               memory_gb=1,
                                               enable_app_insights=True, 
                                               tags={'name': 'aci-cluster', 'framework': 'AutoML'},
                                               description='Heart Failure Predictor Web Service'
                                               )

service = Model.deploy(
    workspace=ws, 
    name=experiment_name+"-service",
    models=[myModel], 
    inference_config=inference_config, 
    deployment_config=aci_config,
    overwrite=True
    )

service.wait_for_deployment(show_output = True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-04-18 08:46:28+00:00 Creating Container Registry if not exists.
2021-04-18 08:46:30+00:00 Use the existing image.
2021-04-18 08:46:30+00:00 Generating deployment configuration.
2021-04-18 08:46:32+00:00 Submitting deployment to compute..
2021-04-18 08:46:37+00:00 Checking the status of deployment heartfailure-automl-service..
2021-04-18 08:49:54+00:00 Checking the status of inference endpoint heartfailure-automl-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [22]:
print(service.scoring_uri)

print(service.swagger_uri)

http://89c1f7e2-618f-4107-8b45-ba27fa5d01ba.southcentralus.azurecontainer.io/score
http://89c1f7e2-618f-4107-8b45-ba27fa5d01ba.southcentralus.azurecontainer.io/swagger.json


In [40]:
import json
import requests

# import test data
test_df = df.sample(5) # data is the pandas dataframe of the original data
label_df = test_df.pop('DEATH_EVENT')

test_sample = json.dumps({'data': test_df.to_dict(orient='records')})
print(test_sample)

{"data": [{"age": 49.0, "anaemia": 0, "creatinine_phosphokinase": 972, "diabetes": 1, "ejection_fraction": 35, "high_blood_pressure": 1, "platelets": 268000.0, "serum_creatinine": 0.8, "serum_sodium": 130, "sex": 0, "smoking": 0, "time": 187}, {"age": 75.0, "anaemia": 0, "creatinine_phosphokinase": 119, "diabetes": 0, "ejection_fraction": 50, "high_blood_pressure": 1, "platelets": 248000.0, "serum_creatinine": 1.1, "serum_sodium": 148, "sex": 1, "smoking": 0, "time": 209}, {"age": 72.0, "anaemia": 1, "creatinine_phosphokinase": 110, "diabetes": 0, "ejection_fraction": 25, "high_blood_pressure": 0, "platelets": 274000.0, "serum_creatinine": 1.0, "serum_sodium": 140, "sex": 1, "smoking": 1, "time": 65}, {"age": 55.0, "anaemia": 1, "creatinine_phosphokinase": 170, "diabetes": 1, "ejection_fraction": 40, "high_blood_pressure": 0, "platelets": 336000.0, "serum_creatinine": 1.2, "serum_sodium": 135, "sex": 1, "smoking": 0, "time": 250}, {"age": 80.0, "anaemia": 0, "creatinine_phosphokinase":

In [41]:
# predict using the deployed model
result = service.run(test_sample)
print(result)

{"result": [0, 0, 1, 0, 1]}


In [42]:
# Set the content type
headers = {'Content-type': 'application/json'}

response = requests.post(service.scoring_uri, test_sample, headers=headers)

# Print results from the inference
print(response.text)

"{\"result\": [0, 0, 1, 0, 1]}"


In [43]:
# Print original labels
print(label_df)

209    0
236    0
67     1
285    0
190    0
Name: DEATH_EVENT, dtype: int64


In [44]:
print(service.get_logs())

2021-04-18T08:49:44,366935100+00:00 - iot-server/run 
2021-04-18T08:49:44,372930000+00:00 - rsyslog/run 
2021-04-18T08:49:44,366742400+00:00 - gunicorn/run 
rsyslogd: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libuuid.so.1: no version information available (required by rsyslogd)
2021-04-18T08:49:44,427573000+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml

In [45]:
service.delete()